In [1]:
import numpy as np
import subprocess
import os
os.chdir(os.path.expanduser('..'))
import shap
import scipy.stats as stats
from multiprocessing import Pool
from utils.rerank import write_average, rerank_ndcg, rerank_matrix,write_tau,write_ratio,write_average2,write_tau2,write_ratio2
from utils.readdata import get_microsoft_data, rewrite
from utils.separate_set import separate_set
from utils.explainer_tools import rand_row, evaluate, get_pairsname, get_rankedduculist,small_get_pairsname, get_set_cover, get_set_cover_beam

In [2]:
def loop_query():
    restore_path = temp_path +  'restore_xfeatures.txt'
    scorefile_path = temp_path + 'scorefile_xfeatures.txt'
    scores = []
    rewrite(X_test, y_query_test, Query_test, restore_path)
    args = ['java', '-jar', 'RankLib-2.12.jar', '-rank', restore_path, '-load', model,
            '-indri', scorefile_path]
    subprocess.check_output(args, stderr=subprocess.STDOUT)
    with open(scorefile_path, 'r') as f:
        for line in f:
            scores.append(float(line.split()[-2]))

    restore_context = open(restore_path, 'r').readlines()

    scores = np.array(scores)
    
    index = 0
    rankedduculist1_set = []
    NDCG_before_set = []
    for i in range(len(q_d_len)):
        this_scores = list(np.array(scores[index:index + q_d_len[i]].reshape(-1, 1)).reshape(-1,1))
        tmp_test_data =test_data[i]
        tmp_test_y_query = test_y_query[i]
        tmp_test_Query = test_Query[i]
        query_id = tmp_test_y_query[0].split(':')[-1].split()[0]

        
        test_data_score = np.append(tmp_test_data,this_scores,axis=1)
        ranked_test_data = np.array((test_data_score[(-test_data_score[:,-1]).argsort()])[:,:-1])
        rankedduculist1 = get_rankedduculist(this_scores, i, q_d_len)
        rankedduculist1_set.append(rankedduculist1)
        if only_tau != 1:
            with open(restore_path, 'w') as f:

                for lineindex in range(q_d_len[i]):
                    split = restore_context[index].split()
                    split[1] = 'qid:{}'.format(y_query_test[index].split(':')[-1].split()[0])
                    newline = ''
                    for i in range(len(split)):
                        newline += (split[i] + ' ')
                    f.write(newline + '\n')
                    index+=1

            NDCG_before = evaluate(model, restore_path)
            NDCG_before_set.append(NDCG_before)
        else:
            index+=q_d_len[i]            
    with open(temp_path+'changed_list_WMATRIX.txt','w') as f:
        m = 0
        for i in range(len(test_data)):
            features_to_change = test_data[i].copy()
            top_k_idx = feature_set[i]
            features_to_change[:,top_k_idx[:feature_number]] = expected_value[top_k_idx[:feature_number]]
            for k in range(len(features_to_change)):
                line = ""
                line += "0 qid:{} ".format(str(m))
                for j in range(len(X_test[0])):
                    line += ((str(j+1))+":"+str(features_to_change[k][j])+" ")
                line += "\n"
                m += 1
                f.write(line)
    args = ['java', '-jar', 'RankLib-2.12.jar', '-rank', temp_path+'changed_list_WMATRIX.txt', '-load', model,
            '-indri', temp_path+'changed_list_WMATRIX_score.txt']
    subprocess.check_output(args, stderr=subprocess.STDOUT)
    A = ''.join(sorted(open(temp_path+'changed_list_WMATRIX_score.txt'), key=lambda s: int(s.split()[0]), reverse=False))
    with open(temp_path+'changed_list_WMATRIX_score.txt','w') as f:
        f.write(A)
    changed_list_score = []
    with open(temp_path+'changed_list_WMATRIX_score.txt','r') as f:
        for line in f:
            changed_list_score.append(float(line.split()[-2]))
    restore_context2 = open(temp_path+'changed_list_WMATRIX.txt', 'r').readlines()
    index2 = 0
    rankedduculist2_set = []
    NDCG_after_set = []
    tau_set = []
    ratio_NDCG_set = []
    
    for i in range(len(q_d_len)):
        this_scores =  list(np.array(changed_list_score[index2:index2 + q_d_len[i]]).reshape(-1, 1))
        tmp_test_data =test_data[i]
        tmp_test_y_query = test_y_query[i]
        tmp_test_Query = test_Query[i]
        query_id = tmp_test_y_query[0].split(':')[-1].split()[0]
    
        test_data_score = np.append(tmp_test_data,this_scores,axis=1)
        ranked_test_data = np.array((test_data_score[(-test_data_score[:,-1]).argsort()])[:,:-1])
        rankedduculist2 = get_rankedduculist(this_scores, i, q_d_len)
        rankedduculist2_set.append(rankedduculist2)
        tau, p_value = stats.kendalltau(rankedduculist1_set[i], rankedduculist2)
        tau_set.append(tau)
        if only_tau != 1:
            with open(restore_path, 'w') as f:
                for lineindex in range(q_d_len[i]):
                    split = restore_context2[index2].split()
                    split[1] = 'qid:{}'.format(y_query_test[index2].split(':')[-1].split()[0])
                    newline = ''
                    for k in range(len(split)):
                        newline += (split[k] + ' ')
                    f.write(newline + '\n')
                    index2+=1
            NDCG_after = evaluate(model, restore_path)
            NDCG_after_set.append(NDCG_after)
            delta_NDCG = abs(float(NDCG_before_set[i]) - float(NDCG_after))
            if float(NDCG_before_set[i])  == 0:
                ratio_NDCG = 0
            else:
                ratio_NDCG = delta_NDCG/float(NDCG_before_set[i]) 
            ratio_NDCG_set.append(ratio_NDCG)
        else:
            index2+=q_d_len[i]
  
        
    complement_idx_set  = []  
    all_features = [i for i in range(len(X_test[0]))]     
    with open(temp_path+'changed_list_WMATRIX.txt','w') as f:
        m = 0
        for i in range(len(test_data)):
            features_to_change = test_data[i].copy()
            top_k_idx = feature_set[i]
            complement_idx = list(set(all_features) - set(top_k_idx[:feature_number]))
            complement_idx_set.append(complement_idx)
            features_to_change[:,complement_idx] = expected_value[complement_idx]
            for k in range(len(features_to_change)):
                line = ""
                line += "0 qid:{} ".format(str(m))
                for j in range(len(X_test[0])):
                    line += ((str(j+1))+":"+str(features_to_change[k][j])+" ")
                line += "\n"
                m += 1
                f.write(line)

    args = ['java', '-jar', 'RankLib-2.12.jar', '-rank', temp_path+'changed_list_WMATRIX.txt', '-load', model,
            '-indri', temp_path+'changed_list_WMATRIX_score.txt']
    subprocess.check_output(args, stderr=subprocess.STDOUT)
    A = ''.join(sorted(open(temp_path+'changed_list_WMATRIX_score.txt'), key=lambda s: int(s.split()[0]), reverse=False))
    with open(temp_path+'changed_list_WMATRIX_score.txt','w') as f:
        f.write(A)
    changed_list_score = []
    with open(temp_path+'changed_list_WMATRIX_score.txt','r') as f:
        for line in f:
            changed_list_score.append(float(line.split()[-2]))
    restore_context3 = open(temp_path+'changed_list_WMATRIX.txt', 'r').readlines()
    index3 = 0
    rankedduculist3_set = []
    NDCG_after2_set = []
    tau2_set = []
    ratio_NDCG2_set = []
    
    for i in range(len(q_d_len)):
        this_scores =list(np.array(changed_list_score[index3:index3 + q_d_len[i]]).reshape(-1, 1))
        tmp_test_data =test_data[i]
        tmp_test_y_query = test_y_query[i]
        tmp_test_Query = test_Query[i]
        query_id = tmp_test_y_query[0].split(':')[-1].split()[0]
    
        test_data_score = np.append(tmp_test_data,this_scores,axis=1)
        ranked_test_data = np.array((test_data_score[(-test_data_score[:,-1]).argsort()])[:,:-1])
        rankedduculist3 = get_rankedduculist(this_scores, i, q_d_len)
        rankedduculist3_set.append(rankedduculist3)
        tau2, p_value = stats.kendalltau(rankedduculist1_set[i], rankedduculist3)
        tau2_set.append(tau2)
        if only_tau != 1:
            with open(restore_path, 'w') as f:

                for lineindex in range(q_d_len[i]):
                    split = restore_context3[index3].split()
                    split[1] = 'qid:{}'.format(y_query_test[index3].split(':')[-1].split()[0])
                    newline = ''
                    for k in range(len(split)):
                        newline += (split[k] + ' ')
                    f.write(newline + '\n')
                    index3+=1

            NDCG_after2 = evaluate(model, restore_path)
            NDCG_after2_set.append(NDCG_after)
            delta_NDCG2 = abs(float(NDCG_before_set[i]) - float(NDCG_after2))
            if float(NDCG_before_set[i])  == 0:
                ratio_NDCG2 = 0
            else:
                ratio_NDCG2 = delta_NDCG2/float(NDCG_before_set[i]) 
            ratio_NDCG2_set.append(ratio_NDCG2)
        else:
            index3+=q_d_len[i]


    for i in range(len(q_d_len)):
        with open(NDCG_file_name, 'a') as NDCG_FILE:
            if only_tau != 1:
                NDCG_line =  test_y_query[i][0].split(':')[-1]+'  ' \
                            + 'changed feature:'+ str(feature_set[i][:feature_number])+' '+'kendalltau='+str(round(tau_set[i],4)) + '  '+'ratioNDCG:'+ str(round(ratio_NDCG_set[i],4)) + \
                            '   ' + 'delta_NDCG ='+'  '+str(round(delta_NDCG_set[i],4))+ "\n"
            else:
                NDCG_line =  test_y_query[i][0].split(':')[-1]+'  ' \
                            + 'changed feature:'+ str(feature_set[i][:feature_number])+' '+'kendalltau='+str(round(tau_set[i],4))+ "\n"
                
            NDCG_FILE.write(NDCG_line)
            
        with open(complement_NDCG_file_name, 'a') as NDCG_FILE:
            if only_tau != 1:
                NDCG_line =  test_y_query[i][0].split(':')[-1]+'  ' \
                            + 'changed feature:'+ str(complement_idx_set[i])+' '+'kendalltau='+str(round(tau2_set[i],4)) + '  '+'ratioNDCG:'+ str(round(ratio2_NDCG_set[i],4)) + \
                            '   ' + 'delta_NDCG ='+'  '+str(round(delta_NDCG2_set[i],4))+ "\n"
            else:
                NDCG_line =  test_y_query[i][0].split(':')[-1]+'  ' \
                            + 'changed feature:'+ str(complement_idx_set[i])+' '+'kendalltau='+str(round(tau2_set[i],4))+ "\n"
                
            NDCG_FILE.write(NDCG_line)    
        with open(ranklist_file, 'a') as ranklist:
            ranklist_line = test_y_query[i][0].split(':')[-1]+ '  ' + 'ranklist before:' + str(
                rankedduculist1_set[i]) + '  ' + 'ranklist after:' + '  ' + str(rankedduculist2_set[i]) + "\n"
            ranklist.write(ranklist_line)
        with open(complement_ranklist_file, 'a') as ranklist:
            ranklist_line = test_y_query[i][0].split(':')[-1]+ '  ' + 'ranklist before:' + str(
                rankedduculist1_set[i]) + '  ' + 'ranklist after:' + '  ' + str(rankedduculist3_set[i]) + "\n"
            ranklist.write(ranklist_line)    

    os.remove(os.path.join(temp_path, 'changed_list_WMATRIX.txt'))
    os.remove(os.path.join(temp_path, 'changed_list_WMATRIX_score.txt')) 
    os.remove(scorefile_path)
    os.remove(restore_path)
             

In [72]:
if __name__ == '__main__':
    #parameters to be set
    dataset = 'mq2008'
    feature_number = 3
    only_tau = 1
    if dataset == 'mq2008':
        model_path = 'model/'
        dataset_path = 'MQ2008/'
        modelname_index = 0
        model_set  =['LambdaMART_model.txt','Ranknet_model.txt','Linearregression_model.txt']
    else:
        model_path = 'MSLR-WEB10K_model/'
        dataset_path = 'MSLR-WEB10K/'
        modelname_index = 1    
        model_set  =['LambdaMART_model.txt','RankBoost_model.txt','LinearRegression_model.txt']

  
    for f in range(1,2):
        # the path of data
        datapath =dataset_path+'Fold{}/'.format(f)
        train_path = datapath + 'train.txt'
        test_path = datapath + 'test.txt'
        dataname = datapath.split('/')[0] +'_'+ datapath.split('/')[1].split('Fold')[1]
        # saving path and save files
        NDCGdata_path = 'logs/'
        temp_path = 'temp_file/'       
        # get train data and test data
        """
        X_train, y_query_train, Query_train = get_microsoft_data(train_path)
        X_train = np.array(X_train)
        X_test, y_query_test, Query_test = get_microsoft_data(test_path)
        X_test = np.array(X_test)
        expected_value = np.mean(X_train, axis=0)
            
        # separate the test set
        test_data, test_y_query, test_Query, q_d_len = separate_set(y_query_test, X_test, Query_test)
        """
        if dataset == 'mslr':
            test_data = test_data[:500]
            test_y_query = test_y_query[:500]
            test_Query = test_Query[:500]
            q_d_len = q_d_len[:500]

      
        for MODEL in model_set:
            model = model_path + MODEL
            modelname = model.split("_")[modelname_index].split("/")[-1]
            old_NDCG_file = 'logs/' +'{}_baseline_5features_threshold0{}.txt'.format(dataname,modelname)
            method_name = old_NDCG_file.split(dataname+'_')[1].split('_')[0]
            resultfile_NDCG = 'resultfile/' + '{}_{}_{}_{}features_NDCG.txt'.format(dataname,method_name,modelname,feature_number)
            resultfile_tau = 'resultfile/' + '{}_{}_{}_{}features_tau.txt'.format(dataname,method_name,modelname,feature_number)
            resultfile_ratio =  'resultfile/' + '{}_{}_{}_{}features_ratio.txt'.format(dataname,method_name,modelname,feature_number)
            complement_resultfile_NDCG = 'resultfile/' + '{}_{}_{}_{}featurescomplement_NDCG.txt'.format(dataname,modelname,method_name,feature_number)
            complement_resultfile_tau = 'resultfile/' + '{}_{}_{}_{}featurescomplement_tau.txt'.format(dataname,modelname,method_name,feature_number)
            complement_resultfile_ratio =  'resultfile/' + '{}_{}_{}_{}featurescomplement_ratio.txt'.format(dataname,modelname,method_name,feature_number)
            
            feature_set = []
            with open(old_NDCG_file,'r') as oldfile:
                for line in oldfile:
                    features = ((line.split(':[')[1].split(']')[0])).split()
                    this_feature_set = [int(features[i].rstrip(',')) for i in range(len(features))]
                    feature_set.append(this_feature_set)
            NDCG_file_name = NDCGdata_path + '{}_{}_{}features'.format(dataname,method_name,feature_number) + modelname + '.txt'
            ranklist_file = NDCGdata_path + '{}_ranklist_{}_{}features'.format(dataname,method_name,feature_number)  + modelname + '.txt'
            complement_NDCG_file_name =  NDCGdata_path + '{}_{}_{}featurescomplement'.format(dataname,method_name,feature_number) + modelname + '.txt'
            complement_ranklist_file = NDCGdata_path + '{}_ranklist_{}_{}features_complement'.format(dataname,method_name,feature_number)  + modelname + '.txt'
            loop_query()

            if only_tau != 1:
                NDCG =  write_average(NDCG_file_name)
                ratio = write_ratio(NDCG_file_name)
                with open(resultfile_NDCG, 'a') as NDCG_result:
                    NDCG_result_line  = str(NDCG) + "\n"
                    NDCG_result.write(NDCG_result_line)
                with open(resultfile_ratio,'a') as ratio_result:
                    ratio_result_line  = str(ratio) + "\n" 
                    ratio_result.write(ratio_result_line)
                NDCG =  write_average(complement_NDCG_file_name)  
                ratio = write_ratio(complement_NDCG_file_name)
                with open(complement_resultfile_NDCG, 'a') as NDCG_result:
                    NDCG_result_line  = str(NDCG) + "\n"
                    NDCG_result.write(NDCG_result_line)
                with open(complement_resultfile_ratio,'a') as ratio_result:
                    ratio_result_line  = str(ratio) + "\n" 
                    ratio_result.write(ratio_result_line)    
            rerank_ndcg(NDCG_file_name)
            rerank_ndcg(ranklist_file)
            tau = write_tau(NDCG_file_name)
            with open(resultfile_tau,'a') as tau_result:
                tau_result_line  = str(tau) + "\n" 
                tau_result.write(tau_result_line)
            rerank_ndcg(complement_NDCG_file_name)
            rerank_ndcg(complement_ranklist_file)
            tau = write_tau(complement_NDCG_file_name)
            with open(complement_resultfile_tau,'a') as tau_result:
                tau_result_line  = str(tau) + "\n" 
                tau_result.write(tau_result_line)